In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps
import PIL
import torch, torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from dataset import defectDataset_convolution
import random
import math
import seaborn
from  matplotlib import pyplot
import torch.nn.functional as F
import matplotlib.image as mpimg
import torch.nn as nn
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import data, img_as_float
import util
from util import *

In [2]:
window_size = 0.15
valid = np.concatenate((np.arange(6501,7001),np.arange(8201,8234),np.arange(8801,8834)))
test = np.concatenate((np.arange(7001,8001),np.arange(8234,8301),np.arange(8834,8901)))
test = np.arange(6501,8001)

In [3]:
df_yolo = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test_yolo.csv', sep=' ')
df_truth = pd.read_csv('/home/rliu/yolo2/v2_pytorch_yolo2/data/an_data/VOCdevkit/VOC2007/csv_labels/test_modified.csv',index_col=0)
df_MMN = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test_MMN.csv', sep=' ')
columns = ['image_index','class', 'x', 'y']
df_yolo_points = pd.DataFrame(index=np.arange(len(df_yolo)), columns=columns)
df_yolo_points['image_index'] = df_yolo['image_index']
df_yolo_points['class'] = df_yolo['class']
df_yolo_points['x'] = (df_yolo['x1'] + df_yolo['x2'])/2
df_yolo_points['y'] = (df_yolo['y1'] + df_yolo['y2'])/2
for index, row in df_yolo.iterrows():
    if (row['class'] == 0) & (row.confidence<0.24):
        df_yolo_points = df_yolo_points.drop(index)
    if (row['class'] == 1) & (row.confidence<0.1):
        df_yolo_points = df_yolo_points.drop(index)
    if (row['class'] == 2) & (row.confidence<0.12):
        df_yolo_points = df_yolo_points.drop(index)
    if (row['class'] == 3) & (row.confidence<0.06):
        df_yolo_points = df_yolo_points.drop(index)
df_MMN = df_MMN.drop(columns = ['confidence'])
new_order = [1,0,2,3]
df_MMN = df_MMN[df_MMN.columns[new_order]]
df_yolo_points = df_yolo_points.replace({'class':2},0)
# df_yolo_points.to_csv('/home/rliu/github/defect_classifier/results/yolo_1-19-18.csv', sep=' ', index=False, header=False)

In [4]:
columns = ['image_index','class', 'x', 'y']
df_results = pd.DataFrame(index=np.arange(0), columns=columns)
# df_truth = df_truth[(df_truth['image_index'] <= 8900) & (df_truth['image_index'] >= 8801)]
# df_test = df_truth.loc[df_truth['image_index'].isin(np.arange(6501,6601))]
for i in test:# detecting local maxima in confidence map to find defects
    columns = ['image_index','class', 'x', 'y']
    df_img = pd.DataFrame(index=np.arange(0), columns=columns)
    
    arr_pos = np.load('/home/rliu/defect_classifier/convolution_test/pos/%06.0f.npy' % i)
    arr_neg = np.load('/home/rliu/defect_classifier/convolution_test/neg/%06.0f.npy' % i)
    arr_pos_o = np.load('/home/rliu/defect_classifier/convolution_test/pos_o/%06.0f.npy' % i)
    arr_nuc = np.load('/home/rliu/defect_classifier/convolution_test/nuc/%06.0f.npy' % i)
    
    coord_pos = peak_local_max(arr_pos, min_distance=15, threshold_abs=0.98, threshold_rel = 0.9).astype(float)
    coord_neg = peak_local_max(arr_neg, min_distance=8, threshold_abs=0.98, threshold_rel = 0.9).astype(float)
    coord_pos_o = peak_local_max(arr_pos_o, min_distance=12, threshold_abs=0.92, threshold_rel = 0.9).astype(float)
    coord_nuc = peak_local_max(arr_nuc, min_distance=12, threshold_abs=0.93, threshold_rel = 0.9).astype(float)
    
    coord_pos[:,0], coord_pos[:,1] = coord_pos[:,0]/150.0, coord_pos[:,1]/150.0
    coord_neg[:,0], coord_neg[:,1] = coord_neg[:,0]/150.0, coord_neg[:,1]/150.0
    coord_pos_o[:,0], coord_pos_o[:,1] = coord_pos_o[:,0]/150.0, coord_pos_o[:,1]/150.0
    coord_nuc[:,0], coord_nuc[:,1] = coord_nuc[:,0]/150.0, coord_nuc[:,1]/150.0
    
    df_pos = pd.DataFrame(index=np.arange(coord_pos.shape[0]), columns=columns)
    df_neg = pd.DataFrame(index=np.arange(coord_neg.shape[0]), columns=columns)
    df_pos_o = pd.DataFrame(index=np.arange(coord_pos_o.shape[0]), columns=columns)
    df_nuc = pd.DataFrame(index=np.arange(coord_nuc.shape[0]), columns=columns)
    
    df_pos['class'] = 0
    df_neg['class'] = 1
    df_pos_o['class'] = 2
    df_nuc['class'] = 3
    
    df_pos['x'], df_pos['y'] = coord_pos[:,1], 1 - coord_pos[:,0]
    df_neg['x'], df_neg['y'] = coord_neg[:,1], 1 - coord_neg[:,0]
    df_pos_o['x'], df_pos_o['y'] = coord_pos_o[:,1], 1 - coord_pos_o[:,0]
    df_nuc['x'], df_nuc['y'] = coord_nuc[:,1], 1 - coord_nuc[:,0]
    
    df_img = df_img.append(df_pos, ignore_index=True)
    df_img = df_img.append(df_neg, ignore_index=True)
    df_img = df_img.append(df_pos_o, ignore_index=True)
    df_img = df_img.append(df_nuc, ignore_index=True)
    
    df_img['image_index'] = i
    df_results = df_results.append(df_img, ignore_index=True)

In [ ]:
df_results_yolo = checkYolo(df_yolo = df_yolo.replace({'class':2},0), df_results = df_results.replace({'class':2},0), conf_thres_pos = 0.001, conf_thres_neg = 0.001, conf_thres_nuc = 0.001, conf_thres_pos_o = 0.001)
df_results_yolo = putBackYOLO(window_size = 0.20, conf_thres_pos = 0.7, conf_thres_neg = 0.8, conf_thres_nuc = 1, conf_thres_pos_o = 0.9, df_yolo = df_yolo, df_results = df_results_yolo.replace({'class':2},0))
df_results_yolo = df_results_yolo.replace({'class':2},0)
df_truth = df_truth.loc[df_truth['image_index'].isin(test)]
df_results_yolo = df_results_yolo.replace({'class':2},0)
df = compareLabels(df_results_yolo, df_truth.replace({'class':2},0), window_size=window_size)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df)

In [ ]:
precision_tot = df.loc[0,'correct_tot'] / df.loc[0,'pred_tot']
precision_pos = df.loc[0,'correct_pos'] / df.loc[0,'pred_pos']
precision_neg = df.loc[0,'correct_neg'] / df.loc[0,'pred_neg']
precision_pos_o = df.loc[0,'correct_pos_o'] / df.loc[0,'pred_pos_o']
precision_nuc = df.loc[0,'correct_nuc'] / df.loc[0,'pred_nuc']

recall_tot = df.loc[0,'correct_tot'] / df.loc[0,'truth_tot']
recall_pos = df.loc[0,'correct_pos'] / df.loc[0,'truth_pos']
recall_neg = df.loc[0,'correct_neg'] / df.loc[0,'truth_neg']
recall_pos_o = df.loc[0,'correct_pos_o'] / df.loc[0,'truth_pos_o']
recall_nuc = df.loc[0,'correct_nuc'] / df.loc[0,'truth_nuc']

accuracy_tot = 2*(precision_tot*recall_tot)/(precision_tot+recall_tot)
accuracy_pos = 2*(precision_pos*recall_pos)/(precision_pos+recall_pos)
accuracy_neg = 2*(precision_neg*recall_neg)/(precision_neg+recall_neg)
accuracy_pos_o = 2*(precision_pos_o*recall_pos_o)/(precision_pos_o+recall_pos_o)
accuracy_nuc = 2*(precision_nuc*recall_nuc)/(precision_nuc+recall_nuc)

precision_pn = (df.loc[0,'correct_pos']+df.loc[0,'correct_neg'])/ (df.loc[0,'pred_pos']+df.loc[0,'pred_neg'])
recall_pn = (df.loc[0,'correct_pos']+df.loc[0,'correct_neg'])/ (df.loc[0,'truth_pos']+df.loc[0,'truth_neg'])
accuracy_pn = 2*(precision_pn*recall_pn)/(precision_pn+recall_pn)

dis_pos_c = df.loc[0,'dis_pos_c'] / df.loc[0,'correct_pos']
dis_neg_c = df.loc[0,'dis_neg_c'] / df.loc[0,'correct_neg']
dis_pos_o_c = df.loc[0,'dis_pos_o_c'] / df.loc[0,'correct_pos_o']
dis_nuc_c = df.loc[0,'dis_nuc_c'] / df.loc[0,'correct_nuc']



print('precision_tot =', precision_tot)
print('precision_pos =', precision_pos)
print('precision_neg =', precision_neg)
print('precision_pos_o =', precision_pos_o)
print('precision_nuc =', precision_nuc)
print('precision_pn =', precision_pn)
print('\n')
print('recall_tot =', recall_tot)
print('recall_pos =', recall_pos)
print('recall_neg =', recall_neg)
print('recall_pos_o =', recall_pos_o)
print('recall_nuc =', recall_nuc)
print('recall_pn =', recall_pn)
print('\n')
print('accuracy_tot =', accuracy_tot)
print('accuracy_pos =', accuracy_pos)
print('accuracy_neg =', accuracy_neg)
print('accuracy_pos_o =', accuracy_pos_o)
print('accuracy_nuc =', accuracy_nuc)
print('\n')
print('dis_pos =', dis_pos_c)
print('dis_neg =', dis_neg_c)
print('dis_pos_o =', dis_pos_o_c)
print('dis_nuc =', dis_nuc_c)
print('\n')
print('accuracy_pn =', accuracy_pn)

In [ ]:
df_results_post = connect_detection(df_results = df_results_yolo, inspect_range = 16, inspect_radius = 0.05, inspect_thres_low = 0.3, inspect_thres_high = 0.6)
df = util.compareLabels(df_results_post, df_truth.replace({'class':2},0), window_size=window_size)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df)

In [ ]:
precision_tot = df.loc[0,'correct_tot'] / df.loc[0,'pred_tot']
precision_pos = df.loc[0,'correct_pos'] / df.loc[0,'pred_pos']
precision_neg = df.loc[0,'correct_neg'] / df.loc[0,'pred_neg']
precision_pos_o = df.loc[0,'correct_pos_o'] / df.loc[0,'pred_pos_o']
precision_nuc = df.loc[0,'correct_nuc'] / df.loc[0,'pred_nuc']

recall_tot = df.loc[0,'correct_tot'] / df.loc[0,'truth_tot']
recall_pos = df.loc[0,'correct_pos'] / df.loc[0,'truth_pos']
recall_neg = df.loc[0,'correct_neg'] / df.loc[0,'truth_neg']
recall_pos_o = df.loc[0,'correct_pos_o'] / df.loc[0,'truth_pos_o']
recall_nuc = df.loc[0,'correct_nuc'] / df.loc[0,'truth_nuc']

accuracy_tot = 2*(precision_tot*recall_tot)/(precision_tot+recall_tot)
accuracy_pos = 2*(precision_pos*recall_pos)/(precision_pos+recall_pos)
accuracy_neg = 2*(precision_neg*recall_neg)/(precision_neg+recall_neg)
accuracy_pos_o = 2*(precision_pos_o*recall_pos_o)/(precision_pos_o+recall_pos_o)
accuracy_nuc = 2*(precision_nuc*recall_nuc)/(precision_nuc+recall_nuc)

precision_pn = (df.loc[0,'correct_pos']+df.loc[0,'correct_neg'])/ (df.loc[0,'pred_pos']+df.loc[0,'pred_neg'])
recall_pn = (df.loc[0,'correct_pos']+df.loc[0,'correct_neg'])/ (df.loc[0,'truth_pos']+df.loc[0,'truth_neg'])
accuracy_pn = 2*(precision_pn*recall_pn)/(precision_pn+recall_pn)

dis_pos_c = df.loc[0,'dis_pos_c'] / df.loc[0,'correct_pos']
dis_neg_c = df.loc[0,'dis_neg_c'] / df.loc[0,'correct_neg']
dis_pos_o_c = df.loc[0,'dis_pos_o_c'] / df.loc[0,'correct_pos_o']
dis_nuc_c = df.loc[0,'dis_nuc_c'] / df.loc[0,'correct_nuc']



print('precision_tot =', precision_tot)
print('precision_pos =', precision_pos)
print('precision_neg =', precision_neg)
print('precision_pos_o =', precision_pos_o)
print('precision_nuc =', precision_nuc)
print('precision_pn =', precision_pn)
print('\n')
print('recall_tot =', recall_tot)
print('recall_pos =', recall_pos)
print('recall_neg =', recall_neg)
print('recall_pos_o =', recall_pos_o)
print('recall_nuc =', recall_nuc)
print('recall_pn =', recall_pn)
print('\n')
print('accuracy_tot =', accuracy_tot)
print('accuracy_pos =', accuracy_pos)
print('accuracy_neg =', accuracy_neg)
print('accuracy_pos_o =', accuracy_pos_o)
print('accuracy_nuc =', accuracy_nuc)
print('\n')
print('dis_pos =', dis_pos_c)
print('dis_neg =', dis_neg_c)
print('dis_pos_o =', dis_pos_o_c)
print('dis_nuc =', dis_nuc_c)
print('\n')
print('accuracy_pn =', accuracy_pn)

In [ ]:
df_results = df_results_yolo
inspect_range = 16
inspect_radius = 0.05
inspect_thres_low = 0.3
inspect_thres_high = 0.6
head_tail = np.append(df_results['image_index'].unique()[0:(int)(inspect_range/2)], df_results['image_index'].unique()[(int)(len(df_results['image_index'].unique()) - inspect_range/2):(int)(len(df_results['image_index'].unique()))])
for index_fr, row_fr in df_results.iterrows():
    print(index_fr)
    if row_fr['image_index'] not in head_tail and row_fr['class'] == 1:
        head = row_fr['image_index']-inspect_range/2
        tail = row_fr['image_index']+inspect_range/2
        # select previous and future detection
        df_future = df_results[df_results['image_index'].between(head, tail, inclusive=True)]
        # select labels from same class
        df_future = df_future[df_future['class'] == row_fr['class']]
        # calculate distance to point
        df_future['distance'] = np.sqrt(np.power((df_future['x'] - row_fr['x']), 2) + np.power((df_future['y'] - row_fr['y']), 2))
        # choose points with location close to the point
        df_future = df_future[df_future['distance'] <= inspect_radius]
        # remove if detection is low in neightborhood region
        if df_future['image_index'].nunique() <= inspect_range * inspect_thres_low:
            df_results = df_results.drop(index_fr)
        if df_future['image_index'].nunique() >= inspect_range * inspect_thres_high:
            if head in df_future['image_index'].unique() and tail in df_future['image_index'].unique():
                for i in range((int)(head), (int)(tail)):
                    if i not in df_future['image_index'].unique():
                        row_new = row_fr
                        row_new['image_index'] = i
                        row_new.name = 10000000
                        df_results = df_results.append(row_new)
df_results = df_results.reset_index(drop = True)